In [ ]:
# !pip install -q datasets peft requests torch bitsandbytes transformers trl accelerate sentencepiece matplotlib wandb

In [ ]:
import os 
import re,math,torch
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset,Dataset,DatasetDict
import transformers 
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from peft import PeftConfig,PeftModel
import wandb

ModuleNotFoundError: No module named 'peft'

In [2]:
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "amirghadami"


RUN_NAME =  "2025-04-08_10.04.00"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

DATASET_NAME = f"{HF_USER}/pricer-data"


MAX_SEQUENCE_LENGTH = 182

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

EPOCHS = 1
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS  =1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE  = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True



In [4]:
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
login(os.environ['HF_TOKEN'], add_to_git_credential=True) 
os.environ["WANDB_API_KEY"] = os.getenv('WANDB_API_KEY')
# wandb.login()

os.environ['WANDB_PROJECT'] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ['WANDB_WATCH'] = "gradients"

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

In [ ]:
if QUANT_4_BIT:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
else: 
    quant_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16
    )

In [10]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code= True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [11]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config = quant_config,
    device_map="auto"
)

base_model.generation_config.pad_token_id = tokenizer.pad_token_id




ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)

# Set the pad_token_id in generation config
model.generation_config.pad_token_id = tokenizer.pad_token_id

# Now generate text
inputs = tokenizer("The future of AI is", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(outputs[0]))

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   1%|1         | 62.9M/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   3%|2         | 126M/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   6%|6         | 73.4M/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   2%|1         | 94.4M/4.92G [00:00<?, ?B/s]